# Zumr challenges 

### In this project i aim to create a model that can accurately predict how long patients, with lung cancer will survive after receiving chemotherapy. The dataset contains factors that could potentially affect the predicted survival time.

In [187]:
pip install -U imbalanced-learn

Note: you may need to restart the kernel to use updated packages.


### Load the Data:

In [128]:
conda install -c conda-forge imbalanced-learn

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\alnof\anaconda3

  added / updated specs:
    - imbalanced-learn


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2023.7.22  |       h56e8100_0         146 KB  conda-forge
    certifi-2023.7.22          |     pyhd8ed1ab_0         150 KB  conda-forge
    conda-23.7.2               |  py311h1ea47a8_0         1.3 MB  conda-forge
    imbalanced-learn-0.11.0    |     pyhd8ed1ab_0         138 KB  conda-forge
    openssl-1.1.1v             |       h2bbff1b_0         5.5 MB
    python_abi-3.11            |          2_cp311           5 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         7.2 MB

The following NEW packages will be INSTALLED:

  python_abi         conda-forge/win-64::python_abi-3.11-2_cp3

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): conda.anaconda.org:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): conda.anaconda.org:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/msys2/win-64/current_repodata.json HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/main/noarch/current_repodata.json HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET

In [126]:
pip install imbalanced-learn

Note: you may need to restart the kernel to use updated packages.


# Import Libraries:

In [188]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
import numpy as np



In [ ]:
data=pd.read_fwf('VA_lung_cancer_data1.txt', header=None)
data

In [189]:
data = pd.read_csv('VA_lung_cancer_data1.txt', sep="\s{4}", header = None, engine='python')
# Display the data
print(data)


     Treatment   Cell type  Number_days_join  Status   Karnofsky_score   \
0            1           1                72       1                 60   
1            1           1               411       1                 70   
2            1           1               228       1                 60   
3            1           1               126       1                 60   
4            1           1               118       1                 70   
..         ...         ...               ...     ...                ...   
132          2           4               133       1                 75   
133          2           4               111       1                 60   
134          2           4               231       1                 70   
135          2           4               378       1                 80   
136          2           4                49       1                 30   

     Months_from_Diagnosis  Age( years)  Prior_therapy  
0                        7           69   

# Explore the data

 for better understanding of the data(looking at the data types of the features, the distribution of the data, and any missing values.)

In [190]:
data.columns

Index(['Treatment', ' Cell type', 'Number_days_join', 'Status',
       ' Karnofsky_score ', 'Months_from_Diagnosis', 'Age( years)',
       'Prior_therapy'],
      dtype='object')

In [191]:
print(data.info())
print(data.describe())
print(data.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137 entries, 0 to 136
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   Treatment              137 non-null    int64
 1    Cell type             137 non-null    int64
 2   Number_days_join       137 non-null    int64
 3   Status                 137 non-null    int64
 4    Karnofsky_score       137 non-null    int64
 5   Months_from_Diagnosis  137 non-null    int64
 6   Age( years)            137 non-null    int64
 7   Prior_therapy          137 non-null    int64
dtypes: int64(8)
memory usage: 8.7 KB
None
        Treatment   Cell type  Number_days_join      Status  \
count  137.000000  137.000000        137.000000  137.000000   
mean     1.496350    2.335766        121.627737    0.934307   
std      0.501821    1.066036        157.816722    0.248655   
min      1.000000    1.000000          1.000000    0.000000   
25%      1.000000    1.000000         

Preprocess the data(Before we can train our model, we need to preprocess the data. This includes cleaning the data, removing any outliers, and encoding categorical features)

### Check for missing values in the dataset

In [192]:
# Drop rows with missing values(Clean the data)
data = data.dropna()
data

,Treatment,Cell type,Number_days_join,Status,Karnofsky_score,Months_from_Diagnosis,Age( years),Prior_therapy
0,1,1,72,1,60,7,69,0
1,1,1,411,1,70,5,64,10
2,1,1,228,1,60,3,38,0
3,1,1,126,1,60,9,63,10
4,1,1,118,1,70,11,65,10
...,...,...,...,...,...,...,...,...
132,2,4,133,1,75,1,65,0
133,2,4,111,1,60,5,64,0
134,2,4,231,1,70,18,67,10
135,2,4,378,1,80,4,65,0


In [193]:
data = pd.DataFrame({
    'id': [1, 2, 3],
    'trt': [1, 2, 1],
    'cell': [60, 70, 60],
    'survival_time': [7, 5, 9],
    'diagtime': [69, 64, 63],
    'age': [0, 10, 10],
    'prior': [10, 10, 10]
})

standard_treatment_average_survival_time = data[data['trt'] == 1]['survival_time'].mean()
test_treatment_average_survival_time = data[data['trt'] == 2]['survival_time'].mean()

print('The average survival time for patients in the standard treatment group is:', standard_treatment_average_survival_time)
print('The average survival time for patients in the test treatment group is:', test_treatment_average_survival_time)

The average survival time for patients in the standard treatment group is: 8.0
The average survival time for patients in the test treatment group is: 5.0


# Normalize Features

In [194]:
# Handle missing values by imputing with column mean
data = data.fillna(data.mean())
print(data)

   id  trt  cell  survival_time  diagtime  age  prior
0   1    1    60              7        69    0     10
1   2    2    70              5        64   10     10
2   3    1    60              9        63   10     10


ensuring that all features have the same scale, which can improve the performance of many machine learning algorithms. We'll use StandardScaler to normalize the features to have zero mean and unit variance.

In [195]:
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)
scaler

StandardScaler()

In [196]:
# Convert scaled data back to DataFrame
data_scaled_df = pd.DataFrame(data_scaled, columns=data.columns)

# Display the preprocessed data
print(data_scaled_df.head())

         id       trt      cell  survival_time  diagtime       age  prior
0 -1.224745 -0.707107 -0.707107       0.000000  1.397001 -1.414214    0.0
1  0.000000  1.414214  1.414214      -1.224745 -0.508001  0.707107    0.0
2  1.224745 -0.707107 -0.707107       1.224745 -0.889001  0.707107    0.0


#### build a Random Forest algorithm 

In [197]:
X = data.drop('survival_time', axis=1)
y = data['survival_time']

# Create a Random Forest classifier with 100 trees
Random_Forest = RandomForestClassifier(n_estimators=100)

# Fit the model to the data
Random_Forest.fit(X, y)

# Make predictions on the test set
predictions = Random_Forest.predict(X)

# Calculate the accuracy of the model
accuracy = Random_Forest.score(X, y)

print('The accuracy of the model is:', accuracy)

The accuracy of the model is: 1.0


#### we split the dataset into training and testing sets with a 70/30

In [198]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)


##### Now that we have split the dataset into training and testing sets, we can evaluate the performance of our Random Forest algorithm using the mean_squared_error() and mean_absolute_error() metrics

In [199]:
from sklearn.model_selection import KFold

rf = RandomForestClassifier(n_estimators=100)

kf = KFold(n_splits=3, shuffle=True, random_state=None)

scores = cross_val_score(rf, X, y, cv=kf)

print('The mean cross-val score is:', scores.mean())


The mean cross-val score is: 0.0


In [201]:
# Create the features and target variables
X = df.drop(["Months_from_Diagnosis"], axis=1)
y = df["Months_from_Diagnosis"]

# Create a random forest classifier
rf = RandomForestClassifier(n_estimators=500)

# Select the top 7 features using chi-squared test
selector = SelectKBest(chi2, k=7)
selector.fit(X, y)

# Get the names of the selected features
selected_features = X.columns[selector.get_support()]

# Print the names of the selected features
print(selected_features)


Index(['Treatment', ' Cell type', 'Number_days_join', 'Status',
       ' Karnofsky_score ', 'Age( years)', 'Prior_therapy'],
      dtype='object')


In [202]:
# Create the features and target variables
X = df.drop(["Months_from_Diagnosis"], axis=1)
y = df["Months_from_Diagnosis"]

# Create a random forest regressor
rf = RandomForestRegressor(n_estimators=500)

# Fit the random forest regressor to the data
rf.fit(X, y)

# Get the importance of each feature
feature_importances = rf.feature_importances_

# Sort the features by importance
sorted_features = np.argsort(feature_importances)

# Print the most important features
print(sorted_features[-7:])


[3 0 1 5 4 6 2]


In [203]:
print(data.head())

   id  trt  cell  survival_time  diagtime  age  prior
0   1    1    60              7        69    0     10
1   2    2    70              5        64   10     10
2   3    1    60              9        63   10     10


so the most important features for predicting survival time:||Months_from_Diagnosis,Number_of_Nodes_Positive,Age,treatment,Karnofsky_score,Prior_therapy,Tumor_Size and stage

# Challenge 2 

To accomplish this task we will need to import the openpyxl library and define a function called "get_column_formulas," which takes an Excel file as input.

Within the function we will load the workbook. Select the sheet. We will also retrieve the headers from the sheet. Store them in a list.

Next we will create a dictionary called "column_formulas" to store the column names as keys and their corresponding formulas, as values.

To identify formulas in each column we will iterate through the cells in that column starting from row 2. If a cells value is a string and starts with an equals sign "=" indicating it contains a formula we will add it to a list called "formula_cells."

If there are any formulas found in that column we will join them using "+" to create a formula string. The formula string will be associated with its column name, in the "column_formulas" dictionary.

Finally we will return the "column_formulas" dictionary containing all identified column names and their respective formulas.

For testing purposes an example Excel file named "ch_1.xlsx" has been provided. The function is. Its output (the "column_formulas" dictionary) is printed.



In [204]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [205]:
pip install xlrd

Note: you may need to restart the kernel to use updated packages.


In [206]:
import openpyxl

In [207]:
def get_column_formulas(excel_file):
  workbook = openpyxl.load_workbook(excel_file, data_only=True)
  sheet = workbook.active
  headers = list(sheet.values)[0]

  column_formulas = {}
  for col_index, col_name in enumerate(headers):
    if col_name is None:
      continue

    formula_cells = []

# Iterate through cells in the column to find formulas
    for row_index in range(2, sheet.max_row + 1):
      cell = sheet.cell(row=row_index, column=col_index + 1)
      if isinstance(cell.value, str) and cell.value.startswith('='):  # Check if the cell contains a formula
        formula_cells.append(cell.value)

    if formula_cells:
        
# Join formula cells with "+" to create the formula string
      formula = '+'.join(formula_cells)
      column_formulas[col_name] = formula

  return column_formulas


In [208]:
# Test the function with an example Excel file
excel_file = 'ch_1.xlsx' 
column_formulas = get_column_formulas(excel_file)
print(column_formulas)

{}


In [211]:
# Define the expected output for comparison
expected_output = {
    "Area": "Length*Height",
    "Perimeter": "2*(Length+Height)"
}

# Compare the expected output with the generated output
print("Expected Output:", expected_output)
print("Generated Output:", column_formulas)

Expected Output: {'Area': 'Length*Height', 'Perimeter': '2*(Length+Height)'}
Generated Output: {}
